In [ ]:
 ## https://www.data.go.kr/data/15083033/fileData.do

In [4]:
import pandas as pd
import math

# --- 설정 ---
# 읽을 CSV 파일 이름 (업로드된 파일로 변경)
csv_file_name = "소상공인시장진흥공단_상권.csv"

# 생성할 SQL 파일 이름 (새 파일명)
sql_file_name = "insert_sanggwon_businesses.sql"

# 생성할 테이블 이름 (새 테이블명)
table_name = "sanggwon_businesses"
# ----------------

def clean_value_mysql(value):
    """
    Python 데이터를 SQL 문자열로 안전하게 변환합니다.
    - None 또는 NaN -> 'NULL'
    - 문자열 -> 작은따옴표로 감싸고, 내부의 작은따옴표 이스케이프
    """
    if pd.isna(value):
        return "NULL"
    elif isinstance(value, str):
        # MySQL은 '를 ''로, \를 \\로 이스케이프합니다.
        cleaned_value = value.replace("\\", "\\\\").replace("'", "''")
        return f"'{cleaned_value}'"
    else:
        # 숫자나 다른 타입은 문자열로 변환 (경도, 위도 처리)
        return str(value)

# 1. CSV 파일 읽기 (한글 인코딩 처리)
try:
    df = pd.read_csv(csv_file_name, encoding='cp949')
except UnicodeDecodeError:
    try:
        df = pd.read_csv(csv_file_name, encoding='utf-8-sig')
    except Exception as e:
        print(f"파일 읽기 오류: {e}")
        df = pd.DataFrame() # 오류 시 빈 데이터프레임
except Exception as e:
    print(f"파일 읽기 오류: {e}")
    df = pd.DataFrame() # 오류 시 빈 데이터프레임

if not df.empty:
    print(f"'{csv_file_name}' 파일 읽기 성공. {len(df)}개 행 발견.")
    
    # 2. CREATE TABLE 쿼리 생성 (매핑된 컬럼으로 변경)
    # 컬럼명 리스트
    columns = ["상호명", "업종명", "주소", "경도", "위도"]
    column_sql = ", ".join(columns)
    
    create_table_sql = f"""
-- 소상공인 상권 업소 테이블 생성 (MySQL용)
DROP TABLE IF EXISTS {table_name};
CREATE TABLE {table_name} (
    상호명 VARCHAR(255) CHARACTER SET utf8mb4,
    업종명 VARCHAR(255) CHARACTER SET utf8mb4,
    주소 VARCHAR(255) CHARACTER SET utf8mb4,
    경도 DOUBLE,
    위도 DOUBLE
);
\n"""

    # 3. SQL 파일 쓰기 시작
    try:
        with open(sql_file_name, 'w', encoding='utf-8') as f:
            # 파일 상단에 데이터베이스 선택 주석 추가
            f.write("-- 사용할 데이터베이스를 선택하세요 (예: USE your_database_name;)\n")
            
            # CREATE TABLE 문 쓰기
            f.write(create_table_sql)
            f.write("\n")
            
            values_batch = []
            
            # INSERT 문 시작 부분
            insert_prefix = f"INSERT INTO {table_name} ({column_sql}) VALUES \n"
            
            # 4. DataFrame을 행별로 순회하며 VALUES 생성 (매핑된 컬럼으로 변경)
            for row in df.itertuples(index=False):
                # .csv 파일의 실제 컬럼명 사용
                cleaned_row = [
                    clean_value_mysql(getattr(row, '상호명')),
                    clean_value_mysql(getattr(row, '상권업종소분류명')),
                    clean_value_mysql(getattr(row, '도로명주소')),
                    clean_value_mysql(getattr(row, '경도')),
                    clean_value_mysql(getattr(row, '위도'))
                ]
                values_batch.append(f"({', '.join(cleaned_row)})")
            
            
            # 5. INSERT 쿼리 파일에 쓰기
            if values_batch:
                # 모든 데이터를 하나의 INSERT 문으로 합쳐서 씀
                f.write(insert_prefix + ",\n".join(values_batch) + ";\n\n")
                
        print(f"'{sql_file_name}' 파일 생성이 완료되었습니다.")
        print(f"총 {len(df)}개의 행이 파일에 저장되었습니다.")

    except Exception as e:
        print(f"SQL 파일 생성 중 오류: {e}")
else:
    print(f"'{csv_file_name}' 파일을 읽지 못했거나 파일이 비어있습니다.")

'소상공인시장진흥공단_상권.csv' 파일 읽기 성공. 107631개 행 발견.
'insert_sanggwon_businesses.sql' 파일 생성이 완료되었습니다.
총 107631개의 행이 파일에 저장되었습니다.
